In [ ]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")

In [ ]:
# import the necessary packages
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Conv2D
from tensorflow.keras.layers import Dropout, Flatten, Dense, Input
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD, RMSprop, Adagrad, Nadam, Adam
from tensorflow.keras.optimizers import RMSprop
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf

In [ ]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
sess_config = ConfigProto()
sess_config.gpu_options.per_process_gpu_memory_fraction = 0.8
sess_config.gpu_options.allow_growth = True
session = InteractiveSession(config=sess_config)

In [ ]:
BASE_PATH = "dataset"
WARMUP_PLOT_PATH = os.path.sep.join(["output", "warmup.png"])

# VALUE INIT
IMG_SIZE = 224
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)
base_learning_rate = 2e-4
BATCH_SIZE = 32
initial_epochs = 30
fine_tune_epochs = 10
#total_epochs = initial_epochs - fine_tune_epochs
total_epochs = fine_tune_epochs

# initialize the list of class label names
CLASSES = ["Bread", "Dairy product", "Dessert", "Egg", "Fried food", "Meat", "Noodles/Pasta", "Rice", "Seafood", "Soup", "Vegetable/Fruit"]

In [ ]:
def plot_training(history, N, plotPath):
    # construct a plot that plots and saves the training history
    plt.style.use("ggplot")
    plt.figure()
    plt.plot(np.arange(0, N), history.history["loss"], label="train_loss")
    plt.plot(np.arange(0, N), history.history["val_loss"], label="val_loss")
    plt.plot(np.arange(0, N), history.history["accuracy"], label="train_acc")
    plt.plot(np.arange(0, N), history.history["val_accuracy"], label="val_acc")
    plt.title("Training Loss and Accuracy")
    plt.xlabel("Epoch #")
    plt.ylabel("Loss/Accuracy")
    plt.legend(loc="lower left")
    plt.savefig(plotPath)

In [ ]:
# derive the paths to the training, validation, and testing
# directories
trainPath = os.path.sep.join([BASE_PATH, "training"])
valPath = os.path.sep.join([BASE_PATH, "validation"])
testPath = os.path.sep.join([BASE_PATH, "evaluation"])

# determine the total number of image paths in training, validation, 
# and testing directories
totalTrain = len(list(paths.list_images(trainPath)))
totalVal = len(list(paths.list_images(valPath)))
totalTest = len(list(paths.list_images(testPath)))

In [ ]:
# initialize the training data augmentation object
train_datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    zoom_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest")

# initialize the validation/testing data augmentation object (which
# we'll be adding mean subtraction to)
val_datagen = ImageDataGenerator()

# define the ImageNet mean subtraction (in RGB order) and set the
# mean subtraction value for each of the data augmentation
# objects
mean = np.array([123.68, 116.779, 103.939], dtype="float32")
train_datagen.mean = mean
val_datagen.mean = mean

In [ ]:
# initialize the training generator
train_batches = train_datagen.flow_from_directory(
    trainPath,
    target_size=(IMG_SIZE, IMG_SIZE),
    color_mode="rgb",
    shuffle=True,
    batch_size=BATCH_SIZE,
    class_mode="categorical")

# initialize the validation generator
validation_batches = val_datagen.flow_from_directory(
    valPath,
    target_size=(IMG_SIZE, IMG_SIZE),
    color_mode="rgb",
    shuffle=False,
    batch_size=BATCH_SIZE,
    class_mode="categorical")

# initialize the testing generator
test_batches = val_datagen.flow_from_directory(
    testPath,
    target_size=(IMG_SIZE, IMG_SIZE),
    color_mode="rgb",
    shuffle=False,
    batch_size=BATCH_SIZE,
    class_mode="categorical")

In [ ]:
# Create the base model from the pre-trained model 
#base_model = VGG16(input_tensor=Input(shape=IMG_SHAPE), weights="imagenet", include_top=False)
base_model = MobileNetV2(input_tensor=Input(shape=IMG_SHAPE), weights="imagenet", include_top=False, layers=tf.keras.layers) # added layers=tf.keras.layers for batch normalization problem: still not working

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Flatten(name="flatten")(x)
# let's add a fully-connected layer
#x = Dense(512, activation="relu")(x) # VGG16
x = Dense(128, activation="relu")(x) # mobilenetv2
x = Dropout(0.5)(x)
# and a logistic layer -- let's say we have config2.CLASSES classes
predictions = Dense(len(CLASSES), activation="softmax")(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False
    #if isinstance(layer, tf.keras.layers.BatchNormalization):
    if isinstance(layer, tf.python.keras.layers.normalization_v2.BatchNormalization):
        layer._per_input_updates = {}

# loop over the layers in the model and show which ones are trainable
# or not
for layer in base_model.layers:
    print("{}: {}".format(layer, layer.trainable))

print('This is the number of trainable weights '
      'after freezing the conv base:', len(model.trainable_weights))

In [ ]:
print("[INFO] compiling model...")
#opt = SGD(lr=base_learning_rate, momentum=0.9, decay=0.01)
opt = SGD(lr=base_learning_rate, momentum=0.9, decay=base_learning_rate/initial_epochs)
#opt = Adam(lr=base_learning_rate, decay=base_learning_rate/initial_epochs)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

# train the head of the network for a few epochs (all other layers
# are frozen) -- this will allow the new FC layers to start to become
# initialized with actual "learned" values versus pure random
print("[INFO] training head...")
history = model.fit(train_batches,
    epochs=initial_epochs,
    validation_data=validation_batches,
    steps_per_epoch=totalTrain // BATCH_SIZE,
    validation_steps=totalVal // BATCH_SIZE)

# reset the testing generator and evaluate the network after
# fine-tuning just the network head
print("[INFO] evaluating after fine-tuning network head...")
test_batches.reset()
predIdxs = model.predict(x=test_batches, steps=(totalTest // BATCH_SIZE) + 1)
predIdxs = np.argmax(predIdxs, axis=1)
print(classification_report(test_batches.classes, predIdxs, target_names=test_batches.class_indices.keys()))
plot_training(history, initial_epochs, WARMUP_PLOT_PATH)
model.save("model", save_format="h5")